In [ ]:
 def strategy(self):
        from itertools import product
        combinations = product(self.entry_signals, self.exit_signals)
        self.sharpe_of_combinations = []
        #-----------------------------STRATEGY FIELD  (DYNAMIC)------------------------------------
        close = self.train_data.close.to_frame().copy()
        close.rename(columns={"close":"price"}, inplace=True)
        for combination in combinations:
            close["last_n_dayhigh_longtrade"] = close.price.rolling(window=combination[0]).max().shift()
            close["last_n_daylow_longtrade"] = close.price.rolling(window=combination[1]).min().shift()
            close["last_n_daylow_shorttrade"] = close.price.rolling(window=combination[0]).min().shift()
            close["last_n_dayhigh_shorttrade"] = close.price.rolling(window=combination[1]).max().shift()
            temp = close.copy()
            temp["long_entry"] = np.where(temp.price.shift() > temp["last_n_dayhigh_longtrade"].shift(), 1, 0)
            temp["long_exit"] = np.where(temp.price.shift() < temp["last_n_daylow_longtrade"].shift(), 1, 0)
            temp["short_entry"] = np.where(temp.price.shift() < temp["last_n_daylow_shorttrade"].shift(), 1, 0)
            temp["short_exit"] = np.where(temp.price.shift() > temp["last_n_dayhigh_shorttrade"].shift(), 1, 0)
            trades = pd.DataFrame()
            trades[["long_entry", "long_exit", "short_entry", "short_exit"]] = temp.loc[(temp.long_entry==1)|(temp.short_entry==1)|(temp.long_exit==1)|(temp.short_exit==1), ["long_entry", "long_exit", "short_entry", "short_exit"]]
            trades = trades[(trades.long_entry.diff()==1)|(trades.short_entry.diff()==1)|(trades.long_exit.diff()==1)|(trades.short_exit.diff()==1)]
            long_entry_dates = []
            long_exit_dates = []
            short_entry_dates = []
            short_exit_dates = []
            long_inpos=False
            short_inpos=False
            for i in range(len(trades)):
                if trades.iloc[i].long_entry==1:
                    if long_inpos==False and short_inpos==False:
                        long_entry_dates.append(trades.index[i])
                        long_inpos=True
                if trades.iloc[i].long_exit==1:
                    if long_inpos==True:
                        long_exit_dates.append(trades.index[i])
                        long_inpos=False
                if trades.iloc[i].short_entry==1:
                    if long_inpos==False and short_inpos==False:
                        short_entry_dates.append(trades.index[i])
                        short_inpos=True
                if trades.iloc[i].short_exit==1:
                    if short_inpos==True:
                        short_exit_dates.append(trades.index[i])
                        short_inpos=False
            if len(long_entry_dates) > len(long_exit_dates):
                long_entry_dates.pop(-1)
            if len(short_entry_dates) > len(short_exit_dates):
                short_entry_dates.pop(-1)
            close["long_position"] = 0
            close["short_position"] = 0
            long_inposition_intervals = []
            short_inposition_intervals = []
            for i in range(len(long_entry_dates)):
                long_inposition_intervals.append([long_entry_dates[i], long_exit_dates[i]])
            for long_inposition_interval in long_inposition_intervals:
                close.loc[long_inposition_interval[0]:long_inposition_interval[1], "long_position"] = 1
            for i in range(len(short_entry_dates)):
                short_inposition_intervals.append([short_entry_dates[i], short_exit_dates[i]])
            for short_inposition_interval in short_inposition_intervals:
                close.loc[short_inposition_interval[0]:short_inposition_interval[1], "short_position"] = 1
            #commission = np.log(1 - 0.001) + np.log(1 - 0.0002)
            close["log_ret"] = np.log(close.price.pct_change() + 1)
            close["strategy_returns"] = (close.log_ret * close.long_position) + (-1 * close.log_ret * close.short_position)
            close["trades"] = (close.long_position + close.short_position).to_frame().diff()
            #close.loc[close.trades!=1, "trades"] = 0
            #close["strategy_returns_wcomissions"] = close.strategy_returns - (close.trades * commission)
            self.sharpe_of_combinations.append(self.evaluate(close, combination))